In [1]:
# wordcloud and boost specific words?
# statsmodels, ordinary least squares, pvalue


In [2]:
import pandas as pd
import numpy as np
from string import punctuation
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


In [3]:
fname = "../data/S1_8Sharktankpitchesdeals.csv"
data = pd.read_csv(fname)
#valuation is a component of the amoutn asked for and the stake offered therefor doesn't need to be included in the machine learning
data = data.drop(columns=["Valuation"])
data = data[data.Deal_Status != 0]
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
5,102,A Perfect Pear,500000,0.15,Female,Food and Beverage,a gourmet food business . a gourmet food business,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
7,102,Classroom Jams,250000,0.10,Male,Children / Education,a musical way to teach students Shakespeare . ...,1,1,0,0,1,1,1,0,Barbara Corcoran,Daymond John,Kevin O'Leary,Robert Herjavec,NaN
12,103,Stress Free Kids,250000,0.20,Female,Children / Education,a line of books that use stress relieving tech...,1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,824,Rumi Spice,250000,0.05,Mixed,Food and Beverage,saffron imported from Afghanistan . Rumi Spice...,1,0,0,1,0,0,0,0,Mark Cuban,NaN,NaN,NaN,NaN
698,824,Thompson Tee,700000,0.07,Male,Fashion / Beauty,undershirts designed to block underarm sweat ....,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
699,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN


In [4]:
data2 = pd.DataFrame(columns=data.columns)
offset = 0;

for i, j in data.iterrows():
    data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
    
    if not pd.isna(data["Deal_Shark 2"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 2"]
    if not pd.isna(data["Deal_Shark 3"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 3"]
    if not pd.isna(data["Deal_Shark 4"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 4"]
    if not pd.isna(data["Deal_Shark 5"][i]):
        data2 = data2.append(data.loc[i].to_dict(), ignore_index=True)
        offset+=1
        data2["Deal_Shark 1"][offset+i] = data.loc[i]["Deal_Shark 5"]


data2 = data2.drop(columns=["Deal_Shark 2","Deal_Shark 3","Deal_Shark 4","Deal_Shark 5"])
data2 = data2[data2.Deal_Status != 0]
data

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
5,102,A Perfect Pear,500000,0.15,Female,Food and Beverage,a gourmet food business . a gourmet food business,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
7,102,Classroom Jams,250000,0.10,Male,Children / Education,a musical way to teach students Shakespeare . ...,1,1,0,0,1,1,1,0,Barbara Corcoran,Daymond John,Kevin O'Leary,Robert Herjavec,NaN
12,103,Stress Free Kids,250000,0.20,Female,Children / Education,a line of books that use stress relieving tech...,1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,824,Rumi Spice,250000,0.05,Mixed,Food and Beverage,saffron imported from Afghanistan . Rumi Spice...,1,0,0,1,0,0,0,0,Mark Cuban,NaN,NaN,NaN,NaN
698,824,Thompson Tee,700000,0.07,Male,Fashion / Beauty,undershirts designed to block underarm sweat ....,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
699,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN


In [5]:
data = data.infer_objects()
data2 = data2.infer_objects()


In [6]:
# Choose model output

# "Deal_Status"
# "Deal_Shark 1"

# "Kevin O'Leary"
# "Barbara Corcoran"
# "Robert Herjavec"
# "Daymond John"
# "Lori Greiner"
# "Mark Cuban"
# "Others"
shark_to_model = "Deal_Shark 1"

In [7]:
# drop rows where shark_to_model is Nan
data=data.dropna(subset=[shark_to_model])
data2=data2.dropna(subset=[shark_to_model])
data = data.reset_index(drop=True)
data2 = data2.reset_index(drop=True)
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,Lori Greiner,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,Mr. Tod's Pie Factory,460000,0.10,Male,Food and Beverage,a pie company,1,0,0,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
2,102,A Perfect Pear,500000,0.15,Female,Food and Beverage,a gourmet food business . a gourmet food business,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
3,102,Classroom Jams,250000,0.10,Male,Children / Education,a musical way to teach students Shakespeare . ...,1,1,0,0,1,1,1,0,Barbara Corcoran,Daymond John,Kevin O'Leary,Robert Herjavec,NaN
4,103,Stress Free Kids,250000,0.20,Female,Children / Education,a line of books that use stress relieving tech...,1,0,0,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,824,Rumi Spice,250000,0.05,Mixed,Food and Beverage,saffron imported from Afghanistan . Rumi Spice...,1,0,0,1,0,0,0,0,Mark Cuban,NaN,NaN,NaN,NaN
378,824,Thompson Tee,700000,0.07,Male,Fashion / Beauty,undershirts designed to block underarm sweat ....,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
379,824,Wallet Buckle,500000,0.10,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,0,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
380,826,Bridal Buddy,75000,0.10,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,1,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN


In [8]:
# Custom tokenizer from class activity 22-1-4
stopwords = stopwords.words( 'english' ) + list(punctuation)
stemmer = PorterStemmer()
# Stemming
punc_list = list(punctuation)
def special_remove(word):
    if len(word)>2:
        return False
    for c in word:
        if c in punc_list:
            return True
    return False

# custom function that overrides default token generation
def custom_tokenizer(text):
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [stemmer.stem(w) for w in words if w not in stopwords+["..."]]
    # further remove words with a special char
    words = [w for w in words if not special_remove(w)]
    return words

activity_vectorizer = TfidfVectorizer(ngram_range=(1,2)
                            ,tokenizer=custom_tokenizer 
                            ,stop_words='english'
                            # ,min_df=2
                            ,strip_accents='unicode'
                            )

activity_vectors = activity_vectorizer.fit_transform(data["Pitched_Business_Desc"])
activity_df = pd.DataFrame(activity_vectors.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ sun,“ sunglass,“ theyr,“ total,“ uv,“ wrap,”,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
activity_vectors2 = activity_vectorizer.fit_transform(data2["Pitched_Business_Desc"])
activity_df2 = pd.DataFrame(activity_vectors2.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df2

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ sun,“ sunglass,“ theyr,“ total,“ uv,“ wrap,”,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
activity_vectorizer.get_feature_names()

['-not',
 '-not simpli',
 '10',
 '10 day',
 '10 net',
 '10,000',
 '10,000 run',
 '10-30',
 '10-30 minut',
 '100',
 '100 food',
 '100 guarante',
 '100 mile',
 '100 natur',
 '100 recycl',
 '100 time',
 '100 zinc',
 '12',
 '12 fashion',
 '123',
 '123 plush',
 '15',
 '15 flavor',
 '150',
 '150 90',
 '2',
 '2 ipad',
 '2-5',
 '2-5 age',
 '2-in-1',
 '2-in-1 spray',
 '2012',
 '2012 pipcorn',
 '20mph',
 '20mph 30mph',
 '21st',
 '21st centuri',
 '23',
 '23 ounc',
 '24',
 '24 mph',
 '2400',
 '2400 expert',
 '2400 sat',
 '26',
 '26 pound',
 '26g',
 '26g protein',
 '270',
 '270 shoe',
 '3',
 '3 design',
 '3 ipad',
 '3 pack',
 '3-d',
 '3-d design',
 '3-in-1',
 '3-in-1 nail',
 '30',
 '30 minut',
 '300,000',
 '300,000 fund',
 '30mph',
 '30mph travel',
 '35',
 '35 univers',
 '360',
 '360 degre',
 '360-degre',
 '360-degre pictur',
 '3d',
 '3d pop-up',
 '3d touch-bas',
 '4',
 '40+',
 '40+ mile',
 '5',
 '5 minut',
 '5-in-1',
 '5-in-1 babi',
 '5-in-1 convert',
 '50',
 '50 calib',
 '50 mph',
 '5g',
 '5g fru

In [11]:
# Choose features

# To include Pitch, comment in next line, and comment out the following line
# To exclude Pitch, comment out next line, and comment in the following line

selected_features = activity_df
selected_features2 = activity_df2
# selected_features = pd.DataFrame([])

selected_features["Gender"]=data["Gender"]
selected_features["Category"]=data["Category"]
selected_features["Amount_Asked_For"]=data["Amount_Asked_For"]
selected_features["Exchange_For_Stake"]=data["Exchange_For_Stake"]
selected_features

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ uv,“ wrap,”,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Food and Beverage,500000,0.15
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Children / Education,250000,0.10
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Children / Education,250000,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Mixed,Food and Beverage,250000,0.05
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,700000,0.07
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,500000,0.10
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10


In [12]:
selected_features2["Gender"]=data2["Gender"]
selected_features2["Category"]=data2["Category"]
selected_features2["Amount_Asked_For"]=data2["Amount_Asked_For"]
selected_features2["Exchange_For_Stake"]=data2["Exchange_For_Stake"]
selected_features2

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,“ uv,“ wrap,”,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Food and Beverage,500000,0.15
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Children / Education,250000,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,700000,0.07
484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,500000,0.10
485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10
486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10


In [13]:
selected_features = pd.get_dummies(selected_features)
selected_features

,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
selected_features2 = pd.get_dummies(selected_features2)
selected_features2



,-not,-not simpli,10,10 day,10 net,"10,000","10,000 run",10-30,10-30 minut,100,...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
selected_features.columns

Index(['-not', '-not simpli', '10', '10 day', '10 net', '10,000', '10,000 run',
       '10-30', '10-30 minut', '100',
       ...
       'Category_Fitness / Sports / Outdoor', 'Category_Food and Beverage',
       'Category_Green / Clean Tech', 'Category_Health / Wellness',
       'Category_Lifestyle / Home', 'Category_Media / Entertainment',
       'Category_Pet Products', 'Category_Software / Tech', 'Category_Travel',
       'Category_Uncertain / Other'],
      dtype='object', length=8653)

In [16]:
model = MultinomialNB(alpha=.0001)

X_smote, y_smote = SMOTE().fit_sample(selected_features, data[[shark_to_model]].values.ravel())
X = X_smote
y = y_smote

# X = selected_features
# y = data[[shark_to_model]].values.ravel()



print(X.shape)
print(y.shape)

(553, 8653)
(553,)


In [17]:
model2 = MultinomialNB(alpha=.0001)

X_smote2, y_smote2 = SMOTE().fit_sample(selected_features2, data2[[shark_to_model]].values.ravel())
X2 = X_smote2
y2 = y_smote2

# X2 = selected_features2
# y2 = data2[[shark_to_model]].values.ravel()



print(X2.shape)
print(y2.shape)

(707, 8653)
(707,)


In [18]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=f_classif, k=6)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfpvalues = pd.DataFrame(fit.pvalues_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

                          Specs      Score        PValue
665                       becom  11.599881  3.038980e-12
7682                      today   9.998915  1.721832e-10
1894                     custom   9.742852  3.291490e-10
8647  Category_Lifestyle / Home   9.443750  7.020348e-10
8126                     vibrat   8.749961  4.076595e-09
3203                       gift   7.164597  2.276913e-07
4395                       live   6.780910  6.019082e-07
5246                        pad   6.751896  6.477864e-07
2925              flavor includ   6.743544  6.616279e-07
446                     assembl   6.693270  7.514119e-07
327                     android   6.679738  7.775911e-07
328               android plush   6.679738  7.775911e-07
667                becom friend   6.679738  7.775911e-07
1597               connect ipad   6.679738  7.775911e-07
1643              content trobo   6.679738  7.775911e-07
3084               friend child   6.679738  7.775911e-07
3088          friend wirelessli

In [19]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=6)
fit = bestfeatures.fit(X2,y2)
dfscores = pd.DataFrame(fit.scores_)
dfpvalues = pd.DataFrame(fit.pvalues_)
dfcolumns = pd.DataFrame(X2.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))  #print 10 best features

                           Specs         Score        PValue
8634            Amount_Asked_For  3.997187e+07  0.000000e+00
8647   Category_Lifestyle / Home  6.034884e+01  3.823225e-11
8636               Gender_Female  4.100699e+01  2.886496e-07
8640  Category_Business Services  3.600000e+01  2.756626e-06
8644  Category_Food and Beverage  3.300885e+01  1.044480e-05
8642   Category_Fashion / Beauty  2.345098e+01  6.586838e-04
990                   breathalyz  1.749797e+01  7.617251e-03
6867        smartphon breathalyz  1.749797e+01  7.617251e-03
8637                 Gender_Male  1.512727e+01  1.928958e-02
8650    Category_Software / Tech  1.500000e+01  2.025672e-02
8646  Category_Health / Wellness  1.400000e+01  2.963616e-02
670                      bed bug  1.353941e+01  3.522642e-02
1039                         bug  1.353941e+01  3.522642e-02
1040                    bug glue  1.353941e+01  3.522642e-02
3251                        glue  1.353941e+01  3.522642e-02
3252                   g

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
model.fit(X_train, y_train.ravel())

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [22]:
y_pred = model.predict(X_test)

In [23]:
pd.DataFrame({"actual": y_test.reshape(-1), "prediction": y_pred.reshape(-1)})

,actual,prediction
0,Robert Herjavec,Robert Herjavec
1,Daymond John,Daymond John
2,Robert Herjavec,Robert Herjavec
3,Kevin O'Leary,Kevin O'Leary
4,Mark Cuban,Daymond John
...,...,...
134,Kevin O'Leary,Kevin O'Leary
135,Barbara Corcoran,Barbara Corcoran
136,Daymond John,Daymond John
137,Daymond John,Daymond John


In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6906474820143885

In [25]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=42)

In [26]:
model2.fit(X_train2, y_train2.ravel())

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [27]:
y_pred2 = model2.predict(X_test2)

In [28]:
pd.DataFrame({"actual": y_test2.reshape(-1), "prediction": y_pred2.reshape(-1)})

,actual,prediction
0,Barbara Corcoran,Barbara Corcoran
1,Lori Greiner,Lori Greiner
2,Lori Greiner,Barbara Corcoran
3,Robert Herjavec,Robert Herjavec
4,Others,Others
...,...,...
172,Lori Greiner,Lori Greiner
173,Others,Others
174,Kevin O'Leary,Kevin O'Leary
175,Barbara Corcoran,Barbara Corcoran


In [29]:
accuracy_score(y_test2,y_pred2)

0.7796610169491526

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
param_grid = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]}
scores = ['precision', 'recall']
model3 = MultinomialNB()

In [31]:
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        model3, param_grid, scoring='%s_macro' % score,verbose=3)
    clf.fit(X_train2, y_train2)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test2, clf.predict(X_test2)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.715, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.744, total=   0.1s
[CV] alpha=1e-05 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ......................... alpha=1e-05, score=0.771, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.789, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.764, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.732, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.731, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.758, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.776, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] .

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] ............................. alpha=1, score=0.020, total=   0.1s
[CV] alpha=1 .........................................................
[CV] ............................. alpha=1, score=0.020, total=   0.1s
[CV] alpha=1 .........................................................
[CV] ............................. alpha=1, score=0.163, total=   0.1s
[CV] alpha=1 .........................................................


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    3.1s finished


[CV] ........................... alpha=100, score=0.020, total=   0.1s
[CV] alpha=100 .......................................................
[CV] ........................... alpha=100, score=0.020, total=   0.1s
[CV] alpha=100 .......................................................
[CV] ........................... alpha=100, score=0.022, total=   0.1s
Best parameters set found on development set:

{'alpha': 1e-05}

Grid scores on development set:

0.756 (+/-0.051) for {'alpha': 1e-05}
0.750 (+/-0.034) for {'alpha': 0.0001}
0.740 (+/-0.022) for {'alpha': 0.001}
0.735 (+/-0.033) for {'alpha': 0.01}
0.631 (+/-0.078) for {'alpha': 0.1}
0.074 (+/-0.131) for {'alpha': 1}
0.021 (+/-0.001) for {'alpha': 10}
0.021 (+/-0.001) for {'alpha': 100}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

                  precision    recall  f1-score   support

Barbara Corcoran       0.57      0.74      0.64        23


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s



[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.765, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.782, total=   0.1s
[CV] alpha=1e-05 .....................................................
[CV] ......................... alpha=1e-05, score=0.745, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.735, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.730, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.755, total=   0.1s
[CV] alpha=0.0001 ....................................................
[CV] ........................ alpha=0.0001, score=0.772, total=   0.1s
[CV] 

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    3.1s finished


In [32]:
# Import ML models from sklearn
from sklearn.linear_model import LogisticRegression # Regression classifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
from sklearn import svm # Support Vector Machine
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Random Forest and Gradient Boosting Classifier
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifier 
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix # Some metrics to check the performance of the models

In [33]:
# Setting parameters for each algorithm - these are tunable to achieve max accuracy

Classifiers = {'LR':LogisticRegression(random_state=10,C=5,max_iter=300, solver='lbfgs')
               ,'DTC':DecisionTreeClassifier(random_state=10,min_samples_leaf=2)
               ,'RF':RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1)
               # ,'GBC':GradientBoostingClassifier(random_state=10,n_estimators=400,learning_rate=0.2)
               ,'SGD':SGDClassifier(loss="hinge", penalty="l2")
               # ,'SVM':svm.SVC(kernel='linear', C=0.1)
               ,'NB':MultinomialNB(alpha=.0001)
               }

In [34]:
Classifiers.keys()

dict_keys(['LR', 'DTC', 'RF', 'SGD', 'NB'])

In [35]:
# Create a pipeline so you can reuse the code

def ML_Pipeline(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred)
    Recall = recall_score(y_test,y_pred)
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20) 
    return clf

In [36]:
# Create a pipeline so you can reuse the code

def ML_Pipeline2(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train2, y_train2.ravel())
    y_pred = clf.predict(X_test2)
    F1 = f1_score(y_test2, y_pred, zero_division=1,average="weighted")
    Accuracy = accuracy_score(y_test2,y_pred)
    Precision = metrics.precision_score(y_test2,y_pred,average="weighted",zero_division=1)
    Recall = recall_score(y_test2,y_pred,average="weighted")
    Confusion_matrix = confusion_matrix(y_test2,y_pred)
    print('Classifier = '+(clf_name))
    print('F1 = '+str(F1))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20) 
    return clf

In [37]:
clf_dict = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred,average="weighted",zero_division=1)
    Recall = recall_score(y_test,y_pred,average="weighted")
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    clf_dict.append({'Classifier': clf_name, 'Accuracy': Accuracy, 'Precision': Precision, 'Recall': Recall})
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)


Classifier = LR
Accuracy = 0.14388489208633093
Precision = 0.8768179700843641
Recall = 0.14388489208633093
[[ 0  0  0  0  0  0 22]
 [ 0  0  0  0  0  0 27]
 [ 0  0  0  0  0  0 15]
 [ 0  0  0  0  0  0 21]
 [ 0  0  0  0  0  0 14]
 [ 0  0  0  0  0  0 20]
 [ 0  0  0  0  0  0 20]]
Classifier = DTC
Accuracy = 0.37410071942446044
Precision = 0.42525458799733623
Recall = 0.37410071942446044
[[ 6  2  1  6  7  0  0]
 [ 5  8  2  5  3  0  4]
 [ 2  2  8  3  0  0  0]
 [ 1  3  3  6  4  1  3]
 [ 5  1  1  2  3  0  2]
 [ 3  2  2  1  1 10  1]
 [ 1  1  1  6  0  0 11]]
Classifier = RF
Accuracy = 0.5467625899280576
Precision = 0.6814550190511206
Recall = 0.5467625899280576
[[14  0  0  4  4  0  0]
 [10  9  0  3  5  0  0]
 [ 3  0  8  2  2  0  0]
 [ 8  0  1 10  2  0  0]
 [ 7  1  0  1  5  0  0]
 [ 2  0  1  3  1 12  1]
 [ 1  0  0  0  1  0 18]]
Classifier = SGD
Accuracy = 0.10071942446043165
Precision = 0.9094249780032089
Recall = 0.10071942446043165
[[ 0  0  0  0 22  0  0]
 [ 0  0  0  0 27  0  0]
 [ 0  0  0  0 15

In [38]:
clf_dict2 = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train2, y_train2.ravel())
    y_pred = clf.predict(X_test2)
    F1 = f1_score(y_test2, y_pred, zero_division=1,average="weighted")
    Accuracy = accuracy_score(y_test2,y_pred)
    Precision = metrics.precision_score(y_test2,y_pred,average="weighted",zero_division=1)
    Recall = recall_score(y_test2,y_pred,average="weighted")
    Confusion_matrix = confusion_matrix(y_test2,y_pred)
    clf_dict2.append({'Classifier': clf_name, 'F1': F1, 'Accuracy': Accuracy, 'Precision': Precision, 'Recall': Recall})
    print('Classifier = '+(clf_name))
    print('F1 = '+str(F1))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)

Classifier = LR
F1 = 0.03238047052871237
Accuracy = 0.13559322033898305
Precision = 0.882792301062913
Recall = 0.13559322033898305
[[ 0  0  0  0 23  0  0]
 [ 0  0  0  0 24  0  0]
 [ 0  0  0  0 25  0  0]
 [ 0  0  0  0 32  0  0]
 [ 0  0  0  0 24  0  0]
 [ 0  0  0  0 21  0  0]
 [ 0  0  0  0 28  0  0]]
Classifier = DTC
F1 = 0.47689083602970644
Accuracy = 0.4745762711864407
Precision = 0.5240009658718259
Recall = 0.4745762711864407
[[15  4  0  1  1  1  1]
 [ 2 15  0  2  4  1  0]
 [ 6  4 14  0  1  0  0]
 [ 6 13  3  6  3  0  1]
 [ 5  8  2  2  7  0  0]
 [ 3  2  0  2  1 12  1]
 [ 1  1  0  6  3  2 15]]
Classifier = RF
F1 = 0.7118688421670128
Accuracy = 0.7005649717514124
Precision = 0.7884979131813343
Recall = 0.7005649717514124
[[22  0  0  0  1  0  0]
 [ 6 16  0  2  0  0  0]
 [ 5  1 19  0  0  0  0]
 [ 9  8  2 12  1  0  0]
 [ 5  3  0  0 16  0  0]
 [ 3  1  0  0  0 17  0]
 [ 4  0  0  2  0  0 22]]
Classifier = SGD
F1 = 0.02516692347200822
Accuracy = 0.11864406779661017
Precision = 0.895432347026716

In [39]:
clf_df = pd.DataFrame(clf_dict2)
clf_df

,Classifier,F1,Accuracy,Precision,Recall
0,LR,0.032380,0.135593,0.882792,0.135593
1,DTC,0.476891,0.474576,0.524001,0.474576
2,RF,0.711869,0.700565,0.788498,0.700565
3,SGD,0.025167,0.118644,0.895432,0.118644
4,NB,0.777922,0.779661,0.785971,0.779661


In [40]:
clf_df['Accuracy'].max()

0.7796610169491526

In [41]:
best_clf = clf_df.loc[clf_df['Accuracy'].idxmax(),'Classifier']
best_clf

'NB'

In [42]:
best_model = ML_Pipeline2(best_clf)
best_model


Classifier = NB
F1 = 0.7779216494187484
Accuracy = 0.7796610169491526
Precision = 0.7859712133440948
Recall = 0.7796610169491526
[[17  2  0  2  1  1  0]
 [ 3 17  1  2  0  0  1]
 [ 2  0 21  1  1  0  0]
 [ 7  0  2 18  4  1  0]
 [ 1  2  1  1 18  0  1]
 [ 0  0  0  0  0 21  0]
 [ 0  0  0  0  1  1 26]]


MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)

In [43]:
import joblib
joblib.dump(best_model, f"{shark_to_model}_model.pkl")

['Deal_Shark 1_model.pkl']

In [44]:
import pickle
pickle.dump(selected_features.columns,open(f"{shark_to_model}_vocab.pkl","wb"))